In [1]:
# LIBRARIES
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

from sklearn import utils
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [2]:
# PARAMETERS
filepath = 'clustered_data/german.csv'
# protectedCatCol = ['sex',]

# protected_Se
# protectedCatCol = 'sex'

In [3]:
# GET DATA
df = pd.read_csv(filepath)


y_cols = ['y_true','y_pred_score','y_pred']
clust_col = ['clust_mask']
X_df =  df.loc[:, ~df.columns.isin(y_cols+clust_col)]

X = X_df.to_numpy()
X_scaled = StandardScaler().fit_transform(X)
mask = df['clust_mask'].to_numpy()
cluster = X[mask]
custer_scaled = X_scaled[mask]
# y_cols+clust_col

In [4]:
# X_scaled[mask]
cluster_mean = np.mean(cluster,axis=0)
cluster_min =np.min(cluster,axis=0)
cluster_max =np.max(cluster,axis=0)
scaled_cluster_mean = np.mean(custer_scaled,axis=0)

In [5]:
center_df = pd.DataFrame()
center_df['i'] = np.absolute(scaled_cluster_mean).argsort()[::-1][:10]
# center_df['i'] = np.absolute(scaled_cluster_mean).argsort()[::-1]
center_df['col'] = X_df.columns[center_df['i']]
center_df['scaled_mean'] = scaled_cluster_mean[center_df['i']]
center_df['mean'] = cluster_mean[center_df['i']]
center_df['min'] = cluster_min[center_df['i']]
center_df['max'] = cluster_max[center_df['i']]
# 
center_df

,i,col,scaled_mean,mean,min,max
0,13,credit_history=A30,4.290893,1.0,1.0,1.0
1,27,purpose=A49,1.666667,0.6,0.0,1.0
2,15,credit_history=A32,-1.088775,0.0,0.0,0.0
3,47,installment_plans=A143,-1.069497,0.4,0.0,1.0
4,45,installment_plans=A141,1.023151,0.5,0.0,1.0
5,6,number_of_credits,0.847786,1.9,1.0,3.0
6,35,employment=A73,0.777817,0.7,0.0,1.0
7,3,investment_as_income_percentage,-0.718076,2.1,1.0,4.0
8,2,credit_amount,0.643894,4988.0,1082.0,14421.0
9,28,savings=A61,0.612372,0.9,0.0,1.0


In [6]:
# LDA
percent_test = 0.33

n_clust = np.sum(mask)
n_nclust = np.sum(np.invert(mask))
lda_X_clust = X_scaled[mask]
lda_y_clust = mask*1
lda_X_nclust = X_scaled[np.invert(mask)]
lda_y_nclust = np.invert(mask)*1


test_n = int(n_clust*percent_test)
train_n = n_clust - test_n
test_i_clust = random.sample(range(n_clust),test_n)
test_i_nclust = random.sample(range(n_nclust),test_n)
train_i_clust = random.sample(range(n_clust),train_n)
train_i_nclust = random.sample(range(n_nclust),train_n)

X_train = np.concatenate((
    np.take(lda_X_clust,train_i_clust,axis=0),
    np.take(lda_X_nclust,train_i_clust,axis=0))
    ,axis=0)
y_train = np.concatenate((
    np.take(lda_y_clust,train_i_clust,axis=0),
    np.take(lda_y_nclust,train_i_nclust,axis=0)),
    axis=0)
X_train, y_train = utils.shuffle(X_train, y_train)

X_test = np.concatenate((
    np.take(lda_X_clust,test_i_clust,axis=0),
    np.take(lda_X_nclust,test_i_nclust,axis=0))
    ,axis=0)
y_test = np.concatenate((
    np.take(lda_y_clust,test_i_clust,axis=0),
    np.take(lda_y_nclust,test_i_nclust,axis=0)),
    axis=0)
X_test, y_test = utils.shuffle(X_test, y_test)

y_train.shape

(14,)

In [7]:
# scaled_cluster_mean

In [8]:
# see what happens if we separate with LDA
# from sklearn.lda import LDA
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA


# X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
# lea = np.array([1, 1, 1, 2, 2, 2])
clf = LDA()
clf.fit(X_train, y_train)
clf.coef_[0,:].shape

coef = clf.coef_[0,:]
coef_df = pd.DataFrame()
coef_df['i'] = np.absolute(coef).argsort()[::-1][:10]
coef_df['col'] = X_df.columns[coef_df['i']]
coef_df['val'] = coef[coef_df['i']]

y_train_pred = clf.predict(X_train)
# (1*least_mask)

acc = accuracy_score(y_train,y_train_pred)
print('Train accuracy:',acc)

y_test_pred = clf.predict(X_test)
# (1*least_mask)

acc = accuracy_score(y_test,y_test_pred)
print('Test accuracy:',acc)
coef_df

pd.crosstab(y_test,y_test_pred, rownames=['Actual'], colnames=['Predicted'])

Train accuracy: 0.8571428571428571
Test accuracy: 0.6666666666666666


Predicted,0,1
Actual,,
0,2,1
1,1,2


In [9]:
# np.unique(y_clust_pred)

In [10]:
X.shape

(330, 59)

In [11]:
# k nearest neigbor 
from sklearn.neighbors import KNeighborsClassifier

k = 3
neigh = KNeighborsClassifier(n_neighbors=k)
neigh.fit(X_train, y_train)

y_train_pred = neigh.predict(X_train)
# (1*least_mask)

acc = accuracy_score(y_train,y_train_pred)
print('Train accuracy:',acc)

y_test_pred = neigh.predict(X_test)
# (1*least_mask)

acc = accuracy_score(y_test,y_test_pred)
print('Test accuracy:',acc)
coef_df

pd.crosstab(y_test,y_test_pred, rownames=['Actual'], colnames=['Predicted'])

Train accuracy: 0.9285714285714286
Test accuracy: 1.0


Predicted,0,1
Actual,,
0,3,0
1,0,3


In [12]:
# k-nearest with forward stepwise
# import numpy.ma as ma

this_X = X_train
k = 3
acc = [0]*this_X.shape[1]

for i in range(this_X.shape[1]):
    # this_X_mask = np.zeros(this_X.shape[1])
    # this_X_mask[i] = 1
    # this_X_mask = this_X_mask==1
    col_i =[i]
    
    # this_train_X = this_X[:,this_X_mask]
    this_train_X = np.take(this_X,col_i,axis=1)
    this_test_X  = np.take(X_test,col_i,axis=1)
    # print(this_train_X.shape)

    neigh = KNeighborsClassifier(n_neighbors=k)
    neigh.fit(this_train_X, y_train)
    y_test_pred = neigh.predict(this_test_X)
    
    acc[i]=accuracy_score(y_test,y_test_pred)

# print(acc)

def forwardStepwise_knearest(X_train,X_test,y_train,y_test,k,p_max,col_i,col_names):
    col_cnt = X_train.shape[1]
    cols_to_try = list(range(col_cnt))
    cols_to_try = list(set(cols_to_try) - set(col_i))
    
    test_acc = [0]*col_cnt
    
    for i in cols_to_try:
    
        temp_col_i = col_i.copy()
        temp_col_i.append(i)
        # print(col_i.copy())
        # print(temp_col_i)
        
        this_train_X = np.take(X_train,temp_col_i,axis=1)
        this_test_X  = np.take(X_test,temp_col_i,axis=1)
        
        neigh = KNeighborsClassifier(n_neighbors=k)
        neigh.fit(this_train_X, y_train)
        y_test_pred = neigh.predict(this_test_X)

        test_acc[i]=accuracy_score(y_test,y_test_pred)
        
    best_acc = max(test_acc)
    best_acc_i = test_acc.index(best_acc)
    
    col_i.append(best_acc_i)
    
    print('best subset:',np.take(col_names,col_i).to_list())
    print('  acc:',best_acc)
    
    if len(col_i)<p_max:
        col_i = forwardStepwise_knearest(X_train,X_test,y_train,y_test,k,p_max,col_i,col_names)
    
    return(col_i)


        
k = 3
p_max = 10
col_i = []
col_names = X_df.columns
col_i = forwardStepwise_knearest(X_train,X_test,y_train,y_test,k,p_max,col_i,col_names)

best subset: ['credit_amount']
  acc: 1.0
best subset: ['credit_amount', 'investment_as_income_percentage']
  acc: 1.0
best subset: ['credit_amount', 'investment_as_income_percentage', 'month']
  acc: 1.0
best subset: ['credit_amount', 'investment_as_income_percentage', 'month', 'age']
  acc: 1.0
best subset: ['credit_amount', 'investment_as_income_percentage', 'month', 'age', 'status=A12']
  acc: 1.0
best subset: ['credit_amount', 'investment_as_income_percentage', 'month', 'age', 'status=A12', 'status=A13']
  acc: 1.0
best subset: ['credit_amount', 'investment_as_income_percentage', 'month', 'age', 'status=A12', 'status=A13', 'credit_history=A30']
  acc: 1.0
best subset: ['credit_amount', 'investment_as_income_percentage', 'month', 'age', 'status=A12', 'status=A13', 'credit_history=A30', 'Unnamed: 0']
  acc: 1.0
best subset: ['credit_amount', 'investment_as_income_percentage', 'month', 'age', 'status=A12', 'status=A13', 'credit_history=A30', 'Unnamed: 0', 'residence_since']
  acc: 1.

In [13]:
# from operator import itemgetter

# results = []
# results.append(
#     {
#         'i': 1,
#         'added_column': 2,
#         'accuracy':  3
#     }
# )
# results.append(
#     {
#         'i': 1,
#         'added_column': 2,
#         'accuracy':  3
#     }
# )
# list(map(itemgetter('i'), results))